### Relevel UKCA data from hydbrid height (orography-following) levels to altitude (relative to the sea level) levels for comparison with ATom

In [1]:
import warnings
from pathlib import Path

import iris
import numpy as np
from iris.experimental import stratify

In [2]:
from util_commons import EXPERIMENTS
from util_mypaths import path_to_processed, path_to_raw

In [3]:
warnings.filterwarnings("ignore", module="iris")

In [4]:
%%capture
from tqdm.notebook import tqdm as tqdm

tqdm().pandas()

In [5]:
# Choose experiment
exp = "FIRE"
path_to_exp = path_to_processed / EXPERIMENTS[exp]

In [6]:
gases_to_relevel = [
    "ch4",
    "c2h6",
    "c3h8",
    "no",
    "no2",
    "hono",
    "ho2no2",
    "hno3",
    "n2o5",
    "pan",
    "ppan",
    "meono2",
    "etono2",
    "nprono2",
    "iprono2",
    "o3",
]

In [7]:
# Load surface_altitude coordinate
orography = iris.load_cube(str(path_to_raw / "um_orography_xnvtj.nc"))
alt_cb = iris.util.squeeze(orography)
alt_coord = iris.coords.AuxCoord(alt_cb.data, standard_name="surface_altitude", units="m")

In [8]:
# Chosen altitude levels for releveling
tgt_levels = np.arange(0, 14000, 100)  # [meters]

In [9]:
for n in tqdm(gases_to_relevel):
    # Load .nc
    cube = iris.load_cube(str(path_to_exp / f"{EXPERIMENTS[exp]}_{n}.nc"))
    # Add surface_altitude coordinate
    cube.add_aux_coord(alt_coord, data_dims=[2, 3])
    # Derive altitude coordinate from surface_altitude
    factory = iris.aux_factory.HybridHeightFactory(
        delta=cube.coord("level_height"),
        sigma=cube.coord("sigma"),
        orography=cube.coord("surface_altitude"),
    )
    cube.add_aux_factory(factory)
    # Relevel model data from level height to target levels
    cube_rlvled = stratify.relevel(cube, cube.coord("altitude"), tgt_levels, axis="level_height")
    cube_rlvled.rename(n)
    # Save releveled data to .nc
    iris.save(cube_rlvled, str(path_to_exp / "releveled" / f"{EXPERIMENTS[exp]}_relvl_{n}.nc"))

  0%|          | 0/16 [00:00<?, ?it/s]

In [10]:
# Old storage method
# cubelist_rh = [cbr_ch4, cbr_c2h6, cbr_c3h8]
# cubelist_ox = [cbr_o3, cbr_oh]
# cubelist_nox = [cbr_no, cbr_no2]
# cubelist_rono2 = [cbr_meono2, cbr_etono2, cbr_nprono2, cbr_iprono2]